<a href="https://colab.research.google.com/github/ekaratnida/Automated_Tools-/blob/keep/Mini-bads3/Module3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
#https://drive.google.com/file/d/1tV3wDsawGcH-XjWXfWnHWGeNbf0Tp46q/view?usp=sharing
fileDownloaded = drive.CreateFile({'id':'1tV3wDsawGcH-XjWXfWnHWGeNbf0Tp46q'})
fileDownloaded.GetContentFile('Supermarket.csv')

In [6]:
df = pd.read_csv('Supermarket.csv')
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
0,200812,20080516,6,9,1,0.01,PRD0902611,CL00241,DEP00086,G00028,D00008,CUST0000767429,XX,OT,994111000667718,S,MM,Small Shop,Grocery,STORE01026,SS,N01
1,200646,20070108,2,16,1,1.61,PRD0901483,CL00236,DEP00084,G00028,D00008,CUST0000419057,XX,PE,994104000413959,M,UM,Top Up,Fresh,STORE01300,LS,S03
2,200812,20080518,1,15,3,0.03,PRD0900841,CL00212,DEP00070,G00022,D00005,CUST0000714492,XX,OA,994111000633261,M,LA,Small Shop,Nonfood,STORE00343,XLS,W02
3,200632,20061005,5,10,1,1.11,PRD0904263,CL00248,DEP00089,G00030,D00009,CUST0000356307,XX,OA,994102600389888,S,MM,Small Shop,Grocery,STORE01007,XLS,W01
4,200707,20070415,1,8,23,0.23,PRD0902611,CL00241,DEP00086,G00028,D00008,CUST0000058615,XX,OT,994105300205367,L,MM,Top Up,Grocery,STORE01794,LS,W01


In [7]:
print(df.shape)
print(df.describe())
print(df.info())

(3158469, 22)
          SHOP_WEEK     SHOP_DATE  ...         SPEND     BASKET_ID
count  3.158469e+06  3.158469e+06  ...  3.158469e+06  3.158469e+06
mean   2.007019e+05  2.006968e+07  ...  1.910266e+00  9.941059e+14
std    6.633683e+01  7.210264e+03  ...  3.413940e+00  3.374392e+09
min    2.006070e+05  2.006041e+07  ...  0.000000e+00  9.941001e+14
25%    2.006360e+05  2.006103e+07  ...  7.700000e-01  9.941030e+14
50%    2.007130e+05  2.007052e+07  ...  1.230000e+00  9.941059e+14
75%    2.007420e+05  2.007121e+07  ...  2.090000e+00  9.941088e+14
max    2.008190e+05  2.008071e+07  ...  2.594900e+03  9.941117e+14

[8 rows x 7 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3158469 entries, 0 to 3158468
Data columns (total 22 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   SHOP_WEEK                 int64  
 1   SHOP_DATE                 int64  
 2   SHOP_WEEKDAY              int64  
 3   SHOP_HOUR                 int64  
 4   QUANTITY 

In [8]:
!pip install pandasql

In [9]:
sql ='''
SELECT CUST_CODE, 
COUNT(DISTINCT BASKET_ID)/COUNT(DISTINCT SHOP_WEEK) AS VISIT_PER_WEEK,
COUNT(DISTINCT SHOP_WEEK) AS TOTAL_WEEK,
COUNT(DISTINCT PROD_CODE) AS TOTAL_PRODUCT,
SUM(SPEND) AS TOTAL_SALES
FROM df
WHERE CUST_CODE IS NOT NULL GROUP BY CUST_CODE
HAVING TOTAL_SALES < 4000
'''
from pandasql import sqldf

output = sqldf(sql)


In [10]:
print(output.head())

        CUST_CODE  VISIT_PER_WEEK  TOTAL_WEEK  TOTAL_PRODUCT  TOTAL_SALES
0  CUST0000000013               1          12             79       160.32
1  CUST0000000055               1          15             41       433.65
2  CUST0000000679               1          12             36        85.63
3  CUST0000001052               1          28             42       260.92
4  CUST0000001058               1           2             18        71.50


In [44]:
x = df.iloc[:,1:-1]
x.head(5)

,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT
0,20080516,6,9,1,0.01,PRD0902611,CL00241,DEP00086,G00028,D00008,CUST0000767429,XX,OT,994111000667718,S,MM,Small Shop,Grocery,STORE01026,SS
1,20070108,2,16,1,1.61,PRD0901483,CL00236,DEP00084,G00028,D00008,CUST0000419057,XX,PE,994104000413959,M,UM,Top Up,Fresh,STORE01300,LS
2,20080518,1,15,3,0.03,PRD0900841,CL00212,DEP00070,G00022,D00005,CUST0000714492,XX,OA,994111000633261,M,LA,Small Shop,Nonfood,STORE00343,XLS
3,20061005,5,10,1,1.11,PRD0904263,CL00248,DEP00089,G00030,D00009,CUST0000356307,XX,OA,994102600389888,S,MM,Small Shop,Grocery,STORE01007,XLS
4,20070415,1,8,23,0.23,PRD0902611,CL00241,DEP00086,G00028,D00008,CUST0000058615,XX,OT,994105300205367,L,MM,Top Up,Grocery,STORE01794,LS


In [45]:
y = df.iloc[:,-1:]
y.head(5)

,STORE_REGION
0,N01
1,S03
2,W02
3,W01
4,W01


In [46]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(x_train.head())

         SHOP_DATE  SHOP_WEEKDAY  ...  STORE_CODE  STORE_FORMAT
35435     20071109             6  ...  STORE02458            MS
3093080   20080325             3  ...  STORE01026            SS
1988089   20060812             7  ...  STORE00638            LS
541535    20060819             7  ...  STORE01166            SS
2438515   20070721             7  ...  STORE00165            SS

[5 rows x 20 columns]


In [31]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(x_train, y_train)
reg.fit(x_train, y_train)
print(reg.coef_)
print(reg.intercept_)
y_pred1 = reg.predict(x_test)
print('Mean squared error: %.2f'
  % mean_squared_error(y_test, y_pred1))

ValueError: ignored

In [ ]:
import sys
import io
import matplotlib.pyplot as plt
import numpy as np

class DisplayLossCurve(object):
  def __init__(self, print_loss=False):
    self.print_loss = print_loss

  """Make sure the model verbose is set to 1"""
  def __enter__(self):
    self.old_stdout = sys.stdout
    sys.stdout = self.mystdout = io.StringIO()
  
  def __exit__(self, *args, **kwargs):
    sys.stdout = self.old_stdout
    loss_history = self.mystdout.getvalue()
    loss_list = []
    for line in loss_history.split('\n'):
      if(len(line.split("loss: ")) == 1):
        continue
      loss_list.append(float(line.split("loss: ")[-1]))
    plt.figure()
    plt.plot(np.arange(len(loss_list)), loss_list)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")

    if self.print_loss:
      print("=============== Loss Array ===============")
      print(np.array(loss_list))
      
    return True

In [ ]:
from sklearn.linear_model import SGDRegressor

reg = SGDRegressor(verbose=1,max_iter=3000,tol=0.00001)
with DisplayLossCurve():
  reg.fit(x_train, y_train)

y_pred2 = reg.predict(x_test)
print('Mean squared error: %.2f'
  % mean_squared_error(y_test, y_pred2))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#print(x_train.iloc[:,3])
#print(y_train.shape)
plt.figure(figsize=(20,7))
plt.scatter(x_train.iloc[:,3], y_train, color = 'red')
#plt.plot(x_train.iloc[:,1:2], np.reshape(train_result.predict([1, 2, 5, 50]),(-1,1)), color = 'blue')
plt.title('Supermarket')
plt.xlabel('Visit per week')
plt.ylabel('Total sales')
plt.show()